In [ ]:
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina' # Make visualizations look good
%matplotlib inline
#%config InlineBackend.figure_format = 'svg' 
import os
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import join
from tqdm import tqdm
import pandas as pd
import pickle as pkl
import dtreeviz
import imodelsx.process_results
from sklearn.tree import plot_tree
import sys
import pybaobabdt
import numpy as np
import viz
import imodelsx
import sklearn.tree
sys.path.append('../experiments/')
results_dir = '/home/chansingh/mntv1/tree-prompt/mar18'

r = imodelsx.process_results.get_results_df(results_dir, use_cached=True)
r = r[r.seed == 1]

In [ ]:
# select a single model
rd = r
dataset_name = 'emotion'
# dataset_name = 'rotten_tomatoes'
rd = rd[rd.dataset_name == dataset_name]
rd = rd[rd.model_name == 'manual_tree']
rd = rd[rd.checkpoint == 'EleutherAI/gpt-j-6B']
rd = rd[rd.verbalizer_num == 0]
rd = rd[rd.prompt_source == 'manual']
rd = rd[rd.num_prompts == 10]
# print(rd.num_prompts)
run_args = rd.iloc[0]
model_name = run_args.model_name

# load the model
print('acc', run_args.accuracy_test, 'depth', run_args.max_depth, 'dset', run_args.dataset_name)
model = pkl.load(open(join(run_args.save_dir_unique, 'model.pkl'), 'rb'))
dt = model
feature_names = run_args.feature_names
feat_names = ['\n'.join(n.split(',')) for n in feature_names]

In [ ]:
# plot baobab
def colors(x):
    if x == 0:
        return 'red'
    else:
        return 'blue'
CLASSES = {
    'emotion': [b'Sad', b'Happy', b'Love', b'Anger', b'Fear', b'Surprise'],
}
# kwargs = {
#     'colormap': colors,
#     'custom_threshold': ['neg', 'pos'],
# }
dt.classes_ = CLASSES.get(dataset_name, [b'Negative', b'Positive'])
kwargs = {}

VERB_EMOTION_0 = {0: ' Sad.', 1: ' Happy.', 2: ' Love.', 3: ' Anger.', 4: ' Fear.', 5: ' Surprise.'}
if dataset_name == 'emotion':
    feats = feat_names
    for k in VERB_EMOTION_0:
        feats = [x.replace(f'_{k}.0', VERB_EMOTION_0[k]) for x in feats]
else:
    feats = feat_names

ax = pybaobabdt.drawTree(
    dt, figsize=(16, 10), dpi=300, ratio=0.55,
    features=feats,
    show_thresholds=False,
    **kwargs
)
plt.savefig(f'tree_{dataset_name}_bao.pdf', bbox_inches='tight')

In [ ]:
# plot sklearn tree
plt.figure(figsize=(8.5, 4.25))
sklearn.tree.plot_tree(dt, feature_names=feat_names, class_names=['neg', 'pos'],
                        impurity=False,
                        precision=2, rounded=True, proportion=True, filled=True, fontsize=5)
plt.savefig(f'tree_{dataset_name}.pdf')
plt.show()